In [6]:
import re
x = open("/home/julian/data/russia/Snezhnaya/SnezhnayaAndrej.TH").read()

In [7]:
def singlesurvey(sname, content):
    svxcontent = [ ]
    if sname != '_null':
        svxcontent.append("*begin %s" % sname)
    for fline in content.split("\n"):
        wds = fline.split()
        if wds[0] in ["extend", "vthreshold", "declination"]:
            pass
        elif wds[0] in ["flags", "date", "units"]:
            svxcontent.append("*"+" ".join(wds))
        elif wds[0] in ["data"]:
            svxcontent.append("*"+" ".join(wds[:7]))
        else:
            svxcontent.append(" ".join(wds[:5]).replace("'", "_").replace("*", "__"))
    if sname != '_null':
        svxcontent.append("*end %s" % sname)
    return svxcontent

    

In [63]:
allfixes = re.findall("fix (\S*?)@(\S*?)\s+(\S+)\s+(\S+)\s+(\S+)", x)
print(allfixes)

[('TABLICHKA', '20111025_vhodn_vl', '43.2647500', '40.7181900', '1980.00'), ('SV0', '20111025_vhodn_vl', '43.2648100', '40.7184200', '1970.00'), ('BANKA', '20111025_vhodn_vl', '43.2545000', '40.6936100', '1473.00'), ('ILLUZ', '20111025_vhodn_vl', '43.2795600', '40.7138100', '2389.00'), ('MEZHEN', '20111025_vhodn_vl', '43.2740800', '40.7087900', '2015.00'), ('P01', '20121203_vhod', '43.2646550', '40.7181600', '1990.00')]


In [110]:
allsurveys = re.findall("survey (.*)\ncenterline\n([\s\S]*?)\nendcenterline\nendsurvey (.*)\n", x)
allequates = re.findall("equate (\S*?)@(\S*?) (\S*?)@(\S*)\n", x)
allfixes = re.findall("fix (\S*?)@(\S*?)\s+(\S+)\s+(\S+)\s+(\S+)", x)

svxcontent = [ "*cs long-lat", "*cs out UTM37N"]
highestfix = max(allfixes, key=lambda X: float(X[4]))
lng0, lat0, alt0 = float(highestfix[3]), float(highestfix[2]), float(highestfix[4])
ent0 = False
svxcontent.append("*declination auto %s %s %s" % (lng0, lat0, alt0))

for st1, s1, lat, long, z in allfixes:
    if s1 != "_null":
        s1 = s1+"."
    else:
        s1 = ""
    svxcontent.append("*fix %s%s %s %s %s" % (s1, st1, long, lat, z))
    if "P01" in st1:
        svxcontent.append("*entrance %s%s" % (s1, st1))
        if not ent0 or float(z) > float(alt0):
            lng0, lat0, alt0 = float(long), float(lat), float(z)
        print(s1, st1)

svxcontent.append("*begin cardinalpoints")
svxcontent.append("*flags surface")
svxcontent.append("*fix landmark_north reference %s %s %s" % (lng0, lat0+0.1, alt0))
svxcontent.append("*fix landmark_south reference %s %s %s" % (lng0, lat0-0.1, alt0))
svxcontent.append("*fix landmark_east reference %s %s %s" % (lng0+0.1, lat0, alt0))
svxcontent.append("*fix landmark_west reference %s %s %s" % (lng0-0.1, lat0, alt0))
svxcontent.append("*end cardinalpoints")
        
svxcontent.append("")
svxcontent.append("")

for sname, content, endsname in allsurveys:
    #print(sname, endsname)
    assert sname == endsname
    svxcontent.extend(singlesurvey(sname, content))
    svxcontent.append("")
    svxcontent.append("")

for st1, s1, st2, s2 in allequates:
    if s1 != "_null":
        s1 = s1+"."
    else:
        s1 = ""
    if s2 != "_null":
        s2 = s2+"."
    else:
        s2 = ""
    svxcontent.append("*equate %s%s %s%s" % (s1, st1, s2, st2))
    
    

20121203_vhod. P01


In [111]:
svxname = "/home/julian/data/russia/snezhnaya.svx"
fout = open(svxname, "w")
fout.write("\n".join(svxcontent))
fout.close()

In [112]:
import subprocess
f3dname = "/home/julian/data/russia/snezhnaya.3d"
y = subprocess.run(["cavern", svxname, "--output="+f3dname], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [96]:
y.stdout
#svxname = "/home/julian/repositories/groundwindow/test.svx"

b'Survex 1.2.30\nCopyright \xc2\xa9 1990-2016 Olly Betts\n/home/julian/data/russia/snezhnaya.svx:30:13: warning: Suspicious compass reading\n TRZ86 TRZ85 -135.00 -4.00 4.80\n             ^~~~~~~\n/home/julian/data/russia/snezhnaya.svx:30: warning: No survey date specified - using 0 for magnetic declination\n TRZ86 TRZ85 -135.00 -4.00 4.80\n/home/julian/data/russia/snezhnaya.svx:70:13: warning: Suspicious compass reading\n DN31X DN30X -75.00 12.00 9.60\n             ^~~~~~\n/home/julian/data/russia/snezhnaya.svx:71:13: warning: Suspicious compass reading\n DN32X DN31X -88.00 25.00 12.90\n             ^~~~~~\n/home/julian/data/russia/snezhnaya.svx:72:13: warning: Suspicious compass reading\n DN33X DN32X -50.00 -18.00 12.87\n             ^~~~~~\n/home/julian/data/russia/snezhnaya.svx:73:13: warning: Suspicious compass reading\n DN34X DN33X -35.00 -27.00 13.40\n             ^~~~~~\n/home/julian/data/russia/snezhnaya.svx:74:13: warning: Suspicious compass reading\n DN35X DN34X -80.00 6.00 1

In [114]:
subprocess.run(["aven", f3dname])


CompletedProcess(args=['aven', '/home/julian/data/russia/snezhnaya.3d'], returncode=0)

In [115]:
fjsname = "/home/julian/repositories/groundwindow/caves/snezhnaya.js"
y = subprocess.run(["python", "../pypothole/parse3ddmp.py", "--3d="+f3dname, "--js="+fjsname], 
                   stdout=subprocess.PIPE)




In [98]:
print(y.stdout.decode())

In [77]:
import math
earthrad = 6378137; 
nyfac = 2*math.pi*earthrad/360; 
nyfac


111319.49079327358

In [81]:
proj(41.713810, 43.279560)[0] - proj(40.713810, 43.279560)[0]
#proj(40.713810, 44.279560)[1] - proj(40.713810, 43.279560)[1]




111319.49079327378